# Rand Index and Jaccard Coefficient

Rand Index and Jaccard are external cluster validation measures to asses cluster quality. They use a series of decisions to produce a measure a number between 0 and 1 (higher values are better). Each pair of points is considered and a decision is made for each pair. Since we consider pairs of points, for a dataset with n points, the **total number of decisions is (n(n-1))/2.** Example: If there are 3 points x1, x2, x3, then the possible combinations are [x1, x2], [x1, x3], [x2, x3], which is (3(3-1))/2 = 3.

The decisions are:
1. Does this pair belong in the same cluster (same class labels)?
2. Does this pair belong in separate clusters (different class labels)?

Example:

Class labels = {A: x1, x2, B: x3, x4}

Cluster assignments = {X1: x1, x2, x3 X2: x4}

To calculate the Rand Index for the above clustering result, we take each pair and check if the clustering decision is correct.

In [2]:
import pandas as pd

df = {'pairs': ['x1, x2', 'x1, x3', 'x1, x4', 'x2, x3', 'x2, x4', 'x3, x4'],
     'decision': ['clustered together?', 'clustered separate?', 'clustered separate?', 'clustered separate?', 'clustered separate?', 'clustered together?']}
df = pd.DataFrame(df)
df

,pairs,decision
0,"x1, x2",clustered together?
1,"x1, x3",clustered separate?
2,"x1, x4",clustered separate?
3,"x2, x3",clustered separate?
4,"x2, x4",clustered separate?
5,"x3, x4",clustered together?


The decisions are based on the class labels. Example: [x1 x2] have the same class label, so they belong together, which means we check if the clustering algorithm placed them together in the same cluster. For [x1 x3], they have different labels, so they belong in different clusters, which means we check if the algorithm placed them in different clusters.

The following are the situations that can happen: 

**True Positive:** A pair of points have been clustered together, and they have the same class label (they belong together).

**True Negative:** A pair of points have been clustered separately, and they have different class labels (they belong in different clusters).

**False Positive:** A pair of points have been clustered together, but they have different class labels (they belong in different clusters).

**False Negative:** A pair of points have been clustered separately, but they have the same class label (they belong together).

In [3]:
df = pd.concat([df, pd.Series(['TP', 'FP', 'TN', 'FP', 'TN', 'FN']).rename('result')], axis=1)
df

,pairs,decision,result
0,"x1, x2",clustered together?,TP
1,"x1, x3",clustered separate?,FP
2,"x1, x4",clustered separate?,TN
3,"x2, x3",clustered separate?,FP
4,"x2, x4",clustered separate?,TN
5,"x3, x4",clustered together?,FN


### Rand Index = (TP + TN) / (TP + TN + FP + FN) = 3/6 = 0.5
### Jaccard = TP / (TP + FP + FN) = 1/4 = 0.25

**Given are 15 instances {x1, . . . , x15} and three classes {A, B, C}**

**The class assignment is as follows: A : {x1, x2, x3, x4, x5}, B : {x6, x7, x8, x9, x10}, C : {x11, x12, x13, x14, x15}.**

**Executing K-Means with (a) K = 3 and (b) K = 5 results in the two (imperfect) clusterings Z1 and Z2:**

**Z1:**

**X1 = {x1, x2, x3}**

**X2 = {x5, x6, x7, x9, x10, x15}**

**X3 = {x4, x8, x11, x12, x13, x14}**

**Z2:**

**X1 = {x1, x2, x3}**

**X2 = {x5, x6, x7}**

**X3 = {x9, x10, x15}**

**X4 = {x8, x11, x12}**

**X5 = {x4, x13, x14}**

**Compute the Rand Index and Jaccard Coefficient of both clusterings**

This time we have 15 points instead of 4, the total number of decisions will be (15(15-1))/2 = 105, which is impossible to calculate by hand, and in most situations, there will be many more points. If we have 100,000 points, the total number of decisions will be close to 5 billion, which is impossible even for a computer. 

There is a faster method of calculating RI and Jaccard, which uses [combinations](https://www.mathsisfun.com/combinatorics/combinations-permutations.html) 

## Combinatorial method:

We will use combinations and fill a confusion matrix to calculate RI and Jaccard

In [16]:
d = {'Same Cluster': ['TP', 'FP', '?'], 'Different Cluster': ['FN', 'TN', '?'], '-': ['Same Class', 'Different Class', '.'],
    '.': ['?', '?', '?']}
cf = pd.DataFrame(d).set_index('-')
cf

,Same Cluster,Different Cluster,.
-,,,
Same Class,TP,FN,?
Different Class,FP,TN,?
.,?,?,?


Steps:

1. Calculate the total number of decisions to make
2. Pick pairs of points where both points in a pair are from the same cluster (TP + FP)
3. Pick pairs of points where both points in a pair are from the same class (TP + FN)
4. Pick pairs of points where both points in a pair are from the same class AND the same cluster (TP)
5. Fill the rest of the confusion matrix to be able to calculate RI and Jaccard


#### Step 1

**Since there are 15 points, the toal number of pairs will be 15C2 = 105**

So there will be 105 decisions in total

In [23]:
d = {'Same Cluster': ['TP', 'FP', '?'], 'Different Cluster': ['FN', 'TN', '?'], '-': ['Same Class', 'Different Class', '.'],
    '.': ['?', '?', 105]}
cf = pd.DataFrame(d).set_index('-')
cf

,Same Cluster,Different Cluster,.
-,,,
Same Class,TP,FN,?
Different Class,FP,TN,?
.,?,?,105


#### Step 2

**To pick pairs of points where both points in a pair are in the same cluster, we pick pairs from the cluster assignments X1, X2, and X3.**

**Since X1 has 3 points, X2 has 6 points, and X3 has 6 points, the total number of pairs will be 3C2 + 6C2 + 6C2**

**True Positives + False Positives = 3C2 + 6C2 + 6C2 = 33**

The reason this equals TP + FP is as follows: We are only picking pairs from the same cluster; i.e. we are never picking a point from X1 and another from X2. If one point is from X1, the other point in the pair is also from X1, not X2 or X3. Since we are picking pairs that are clustered together, we are inline with the first part of the definition of True Positives (**pair clustered together** and have the same label). The second part of the definition says that they should have the same label to be called a TP. Now some of the pairs that we pick will have the same label, which means they will be True Positives, while some of the pairs will have different labels, which corresponds to the definition of False Positives (pair clustered together but belong separately, i.e. different labels). So this process of picking pairs of points from the cluster assignment gives us TP + FP

In [25]:
d = {'Same Cluster': ['TP', 'FP', 33], 'Different Cluster': ['FN', 'TN', '?'], '-': ['Same Class', 'Different Class', '.'],
    '.': ['?', '?', 105]}
cf = pd.DataFrame(d).set_index('-')
cf

,Same Cluster,Different Cluster,.
-,,,
Same Class,TP,FN,?
Different Class,FP,TN,?
.,33,?,105


#### Step 3

**To pick pairs of points where both points in a pair are in the same class, we pick pairs from the class labels A, B, and C.**

**Since A, B, and C have 5 points each, the total number of pairs will be 5C2 + 5C2 + 5C2**

**True Positives + False Negatives = 5C2 + 5C2 + 5C2 = 30**

The reason this equals TP + FN is as follows: We are only picking pairs from the same class; i.e. we are never picking a point from class A and another from B. If one point is from A, the other point in the pair is also from A, not B or C. Since we are picking pairs that have the same label, we are inline with the second part of the definition of True Positives (pair clustered together and **have the same label**). The second part of the definition says that they should be in the same cluster to be called a TP. Now some of the pairs that we pick will be in the same cluster, which means they will be True Positives, while some of the pairs will be in different clusters, which corresponds to the definition of False Negatives (pair clustered separately but belong together, i.e. same label). So this process of picking pairs of points from the class labels gives us TP + FN

In [26]:
d = {'Same Cluster': ['TP', 'FP', 33], 'Different Cluster': ['FN', 'TN', '?'], '-': ['Same Class', 'Different Class', '.'],
    '.': [30, '?', 105]}
cf = pd.DataFrame(d).set_index('-')
cf

,Same Cluster,Different Cluster,.
-,,,
Same Class,TP,FN,30
Different Class,FP,TN,?
.,33,?,105


#### Step 3

**To pick pairs of points where both points in a pair are in the same class and also clustered together (Example: x1 and x2 are from class A and are in cluster X1), we will pick pairs of points from each cluster, but both points in the pairs must have the same class label. These are the class labels in the cluster assignments:**

X1 = {A A A}

X2 = {A B B B B C}

X3 = {A B C C C C}

For X1, the three points from class A will make 3C2 pairs

For X2, the 4 points from class B will make 4C2 pairs

For X3, the 4 points from class C will make 4C2 pairs

**So, True Positives = 3C2 + 4C2 + 4C2 = 15**

In [30]:
d = {'Same Cluster': [15, 18, 33], 'Different Cluster': [15, 57, '?'], '-': ['Same Class', 'Different Class', '.'],
    '.': [30, '?', 105]}
cf = pd.DataFrame(d).set_index('-')
cf

,Same Cluster,Different Cluster,.
-,,,
Same Class,15,15,30
Different Class,18,57,?
.,33,?,105


### Rand Index = (TP + TN) / (TP + TN + FP + FN) = (15 + 57) / (15+57+15+18) = 0.686
### Jaccard = TP / (TP + FP + FN) = 15 / (15 + 18 + 15) = 0.313